In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.0-bin-hadoop3.2')
DATA_PATH = '../data/Spark_for_Machine_Learning/Spark Streaming/'

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [6]:
spark = SparkSession.builder.appName('StructStream').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/31 12:37:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
## Example from docs
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

21/12/31 12:37:34 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [10]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


# Write data to this stream from terminal by running `nc -lk 9999`
query.awaitTermination()

21/12/31 12:39:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3e5bc460-0955-48b7-93b1-8eff8fa3ea52. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/31 12:39:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
| hello|    2|
|hello?|    1|
+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|    hello|    2|
|burritos?|    1|
|   hello?|    1|
+---------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|      eat|    1|
|     lots|    1|
|    hello|    2|
|   should|    1|
|burritos?|    1|
|    them/|    1|
|       of|    1|
|    them?|    1|
|       we|    1|
|   hello?|    1|
+---------+-----+



KeyboardInterrupt: 